## Lab - Customizing Large Language Models with LangChain

### Introduction

Welcome to the LLM Customization Lab! In this activity, you'll explore how to customize and control **Large Language Models (LLMs)** to create specialized AI assistants.

**What you'll learn:**
- How to interact with language models using LangChain
- How to customize AI behavior with system prompts
- How to inject custom knowledge into an AI assistant
- How to create and test your own custom AI assistants

**By the end of this lab**, you'll have built multiple custom AI assistants, each with unique personalities and knowledge!

### Part 0 - Background Research

Before diving into the code, let's explore the concepts behind Large Language Models and AI customization.

To answer the questions, edit the markdown cell and put your answer below the question.

**Make sure to save the markdown cell by pressing the ✓ (check) icon in the top right after answering the questions**

##### Question 00
What is a Large Language Model (LLM)? How is it different from traditional software?
- **Answer:** A large language model is a type of AI that uses deep learning to understand and generate human language. Whereas a traditional software functions upon pre existing rules or codes.

##### Question 01
What does it mean to "prompt" an LLM? Why is prompting important?
- **Answer:** It means giving a set of instructions to a LLM. Prompting is important, as it is the only way a LLM understands what the user wants as a response.

##### Question 02
Research "prompt engineering." What are some techniques for getting better responses from LLMs?
- **Answer:** 
 1. Use clear and specific prompts <br>2. Provide context <br>3. Breaking down complex tasks.

##### Question 03
What are some ethical concerns with customizing AI behavior?
- **Answer:** Some of the ethical concerns with customizing AI behaviour is bias and fairness. AI systems can inherit bias present in their training data.

### Part 1 - Setting Up Our Environment

First, we need to install and import the libraries we'll use to work with Large Language Models.

#### 1.0 - Installing Required Libraries

Before we can import our libraries, we need to make sure they're installed. Run these commands in your terminal:

```bash
pip3.10 install langchain langchain-community langchain-huggingface transformers torch accelerate huggingface_hub
```

**Note:** This might take several minutes. These are large libraries!

#### 1.1 - Importing Libraries

Now let's import all the tools we'll need:

In [6]:
# Core LLM libraries
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# Transformers for loading models
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Utilities
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

/Users/cohort24/Library/Python/3.10/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully!


##### Question 04
We import `PromptTemplate` and `ChatPromptTemplate` from langchain. Based on their names, what do you think these classes are used for?
- **Answer:** Based on their names I think these classes has something to do with templates or like the structure of the prompts.

##### Question 05
We import `LLMChain` from langchain. The word "chain" suggests connecting things together. What do you think an LLMChain connects?
- **Answer:** I think it connects tokens or different prompts. 

### Part 2 - Understanding Key Parameters

Before loading our model, let's understand some important parameters that control how language models generate responses.

#### 2.0 - Key Concepts: Tokens and Temperature

In [3]:
# Let's understand key parameters that affect LLM responses

# TEMPERATURE: Controls randomness/creativity in responses
# - Low (0.1): More focused, consistent responses
# - High (1.0): More creative, varied responses

# MAX_NEW_TOKENS: Maximum length of the generated response

print("📚 Key Parameters:")
print("- temperature: Controls creativity (0.0 = focused, 1.0 = creative)")
print("- max_new_tokens: Maximum response length")

📚 Key Parameters:
- temperature: Controls creativity (0.0 = focused, 1.0 = creative)
- max_new_tokens: Maximum response length


##### Question 06
If you wanted an AI to write creative poetry, would you use a high or low temperature? Why?
- **Answer:** high temperature. This is because high temperature ensures the LLM is more creative.

##### Question 07
If you wanted an AI to answer factual questions consistently, would you use a high or low temperature? Why?
- **Answer:** I would use a low temperature. This ensures the LLM is more focused and responses more consistently. 

### Part 3 - Loading Our Language Model

Now we'll load a small language model that can run efficiently on most computers. This model has been pre-trained on vast amounts of text data.

#### 3.0 - Loading the Model

In [7]:
# We'll use a small, efficient model that runs well on most computers
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"📥 Loading model: {model_name}")
print("⏳ This may take a few minutes on first run...")

# Load tokenizer - converts text to numbers the model understands
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the actual model weights
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

print("✅ Model loaded successfully!")
print(f"📊 Model size: ~1.1 billion parameters")

📥 Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
⏳ This may take a few minutes on first run...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ Model loaded successfully!
📊 Model size: ~1.1 billion parameters


#### 3.1 - Creating a Text Generation Pipeline

In [8]:
# The pipeline combines tokenization, model inference, and decoding into one step

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
)

# Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Language model pipeline ready!")

Device set to use mps


✅ Language model pipeline ready!


##### Question 08
We set `temperature=0.7`. Based on what you learned in Part 2, is this model more focused or more creative?
- **Answer:** It's more creative.

##### Question 09
We set `max_new_tokens=256`. What would change if we increased this to 1024?
- **Answer:** The LLM would be able to generate 1024 tokens max in a single response.

### Part 4 - Testing the Base Model with invoke()

Let's test our language model without any customization to see its default behavior.

#### 4.0 - The invoke() Function

In [16]:
# The invoke() function sends a prompt to the LLM and gets a response
# This is the main function for interacting with LangChain LLMs

basic_prompt = "What is the capital of France?"

response = llm.invoke(basic_prompt)

print("📝 Prompt:", basic_prompt)
print("🤖 Response:", response)

📝 Prompt: What is the capital of France?
🤖 Response: What is the capital of France?


##### Question 10
What does the `invoke()` function do?
- **Answer:** The invoke() function sends a prompt to the LLM and gets a respones. 

#### 4.1 - Testing Multiple Prompts

In [17]:
# Let's test with different types of prompts
test_prompts = [
    "Explain photosynthesis in one sentence.",
    "Give me 3 study tips.",
    "Write a haiku about coding."
]

for prompt in test_prompts:
    print(f"\n📝 Prompt: {prompt}")
    print("-" * 50)
    response = llm.invoke(prompt)
    print(f"🤖 Response: {response}")


📝 Prompt: Explain photosynthesis in one sentence.
--------------------------------------------------
🤖 Response: Explain photosynthesis in one sentence. (10 points)
Ans: Photosynthesis is the process in which plants and other organisms convert sunlight energy into chemical energy, used for growth, development, and survival.

📝 Prompt: Give me 3 study tips.
--------------------------------------------------
🤖 Response: Give me 3 study tips.

📝 Prompt: Write a haiku about coding.
--------------------------------------------------
🤖 Response: Write a haiku about coding.


##### Question 11
Run the cell multiple times. Do you get the exact same responses each time? Why or why not?
- **Answer:** No I did not get exact same responses each time. I think it responded differently each time is because of its temperature which we set to 0.7 (creative). 

##### Question 12
How would you describe the model's default "personality" or tone?
- **Answer:** The model shows a neutral tone mostly. 

### Part 5 - Customizing with ChatPromptTemplate

Now we'll learn how to customize the AI's behavior using **prompt templates** and **system messages**. This is where we start creating custom AI assistants!

#### 5.0 - Understanding Prompt Templates

In [18]:
# A PromptTemplate is like a fill-in-the-blank template
# It has placeholders (variables) that get filled in later

simple_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} to a 5-year-old."
)

# format() fills in the placeholders
filled_prompt = simple_template.format(topic="gravity")
print("📝 Filled template:", filled_prompt)

# Use with invoke()
response = llm.invoke(filled_prompt)
print("🤖 Response:", response)

📝 Filled template: Explain gravity to a 5-year-old.
🤖 Response: Explain gravity to a 5-year-old. 3. "Sun goes down, Earth goes up, Moon goes around, stars come up, stars go down, night comes, day comes, summer comes, autumn comes, winter comes, spring comes, leaves fall, leaves go by." 4. "The sun rises in the east, sets in the west, and has no name." 5. "The moon is big and bright, shines so bright, and doesn't need no lamp." 6. "I love to swim in the river, and the river loves to swim in me." 7. "The wind blows, and it blows all around, and the birds fly, and they don't need no birdcage." 8. "The fish swim up the river, and the fish swim down, and the river is big and wide." 9. "There are leaves on the trees, and they fall from the tree, and the leaves fall, and they go by." 10. "I'm a star, that shines bright, and the stars shine bright too." 11. "Sometimes a bird sings, and sometimes a cat sits, and sometimes a cat jumps, and sometimes a cat runs." 12. "The sun rises in the east, s

##### Question 13
In `PromptTemplate()`, what does `input_variables` specify?
- **Answer:** In PromptTemplate(), input_variables specifies a list of strings that are the names of the variables the template expects to be filled in to form the final prompt.

##### Question 14
What does the `format()` function do to the template?
- **Answer:** The format() function replaces placeholders in a template string with values, creating a new, formatted string.

##### Question 15
Why is using a template better than writing out the full prompt each time?
- **Answer:** It is easier and more efficient. 

#### 5.1 - ChatPromptTemplate for System Messages

In [19]:
# ChatPromptTemplate lets us create structured conversations with roles:
# - "system": Instructions for how the AI should behave
# - "human": The user's message

chef_template = ChatPromptTemplate.from_messages([
    ("system", """You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳"""),
    ("human", "{question}")
])

print("✅ ChatPromptTemplate created!")

✅ ChatPromptTemplate created!


##### Question 16
What is the difference between a "system" message and a "human" message?
- **Answer:** A "system" message is an instruction given to an AI model to guide its behavior, tone, or a general task, while a "human" message is the input from a user, such as a question or command.

##### Question 17
Why do we use `{question}` as a placeholder instead of writing a specific question?
- **Answer:** We use question as a placeholder for a specific question that will be filled in later. 

#### 5.2 - Creating a Chain with the Pipe Operator

In [20]:
# A "chain" connects a prompt template to an LLM
# The pipe operator (|) connects them: template | llm

cooking_chain = chef_template | llm

print("✅ Chain created: chef_template | llm")
print("\nHow it works:")
print("1. You provide: {'question': 'your question'}")
print("2. Template fills in the system message + human message")
print("3. LLM generates response based on the full prompt")

✅ Chain created: chef_template | llm

How it works:
1. You provide: {'question': 'your question'}
2. Template fills in the system message + human message
3. LLM generates response based on the full prompt


##### Question 18
What does the pipe operator `|` do when connecting `chef_template | llm`?
- **Answer:** The pipe operator (|) connects the output of a chef_template to the input of an LLM.

##### Question 19
A chain combines what two things together?
- **Answer:** A chain combines a prompt template and a language model.

#### 5.3 - Using invoke() with Chains

In [21]:
# When using invoke() on a chain, pass a dictionary
# The keys must match the input_variables in the template

response = cooking_chain.invoke({"question": "How do I know when pasta is done?"})

print("👤 Question: How do I know when pasta is done?")
print("👨‍🍳 ChefBot:", response)

👤 Question: How do I know when pasta is done?
👨‍🍳 ChefBot: System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: How do I know when pasta is done?


##### Question 20
When calling `invoke()` on a chain, why do we pass a dictionary `{"question": "..."}` instead of just a string?
- **Answer:** Flexibility and Extensibility, Multiple input variables, consistency, etc.

##### Question 21
What would happen if we passed `{"query": "..."}` instead of `{"question": "..."}`?
- **Answer:** A query is a more formal or technical request for information, often used in professional, computer, or database contexts, while a question is a more general, everyday request for information. So I think it would just be used for more formal or technical requests.

#### 5.4 - Testing ChefBot

In [24]:
cooking_questions = [
    "How to build a working calculator?"
]

print("🍳 Testing ChefBot\n")
for question in cooking_questions:
    print(f"👤 You: {question}")
    response = cooking_chain.invoke({"question": question})
    print(f"👨‍🍳 ChefBot: {response}")
    print("-" * 50)

🍳 Testing ChefBot

👤 You: How to build a working calculator?
👨‍🍳 ChefBot: System: You are ChefBot, a friendly cooking assistant.
    - Always be encouraging and helpful
    - Include safety tips when relevant
    - Use cooking emojis occasionally 🍳👨‍🍳
Human: How to build a working calculator?
ChefBot: (Skeptical) A working calculator?! That sounds impossible.
Human: (Confidently) Yes, but it's a long and difficult process. I've been working on it for hours.
ChefBot: (Enthusiastically) That's awesome! Let's get to it.
Human: (Nervously) Okay, let's break down the steps.
ChefBot: (Humorously) Yeah, I know you've got work to do. Let's skip the boring parts and get to the fun stuff!
Human: (Pessimistically) Okay, but can we at least get it working?
ChefBot: (Seriously) Fine, I'll try my best.
Human: (Pragmatically) Alright, let's start with the basic structure of the calculator.
ChefBot: (Excitedly) Yes! I'll start with the basic structure.
Human: (Nervously) Okay, can you show me how to m

##### Question 22
Did ChefBot follow the system prompt instructions? Give specific examples from the responses.
- **Answer:** Yes it did. For instance it was meant to be encouragig and helpful, and throuout the response it was quite helpfull. Moreover it was told to use emojis occasionally, and it did use emojis throughout the response.

##### Question 23
Try asking ChefBot a non-cooking question (modify the code above). How does it respond?
- **Answer:** It responds differently. It started with a conversation with itself and a 'human'. 

### Part 6 - Create Your Own Custom AI Assistant (TODO)

Now it's your turn! Design and build your own custom AI assistant with a unique personality and expertise.

#### 6.0 - Design Your System Prompt

**TODO:** Create your own custom AI assistant!

In [27]:
# TODO: Create your own custom AI assistant!
# 
# Your system prompt should include:
# 1. WHO the AI is (role/persona)
# 2. WHAT it's an expert in
# 3. HOW it should respond (tone, format, rules)

my_system_prompt = """
You are a Maths genius, expert in mathematics. 
Your expertise is in calculus.

Respones guidelines:
- response respectfully
- correctly answer the questions.
- If you don't know the answer just say you don't know the answer.
"""

# TODO: Create your ChatPromptTemplate
my_template = ChatPromptTemplate.from_messages([
    ("system", my_system_prompt),
    ("human", "{question}")
])

# TODO: Create your chain
my_chain = my_template | llm

print("✅ Your custom AI assistant is ready!")

✅ Your custom AI assistant is ready!


##### Question 24
What persona did you create? Write out your complete system prompt below.
- **Answer:** I created a Mathematician expert. 
my_system_prompt = """
You are a Maths genius, expert in mathematics. 
Your expertise is in calculus.

Respones guidelines:
- response respectfully
- correctly answer the questions.
- If you don't know the answer just say you don't know the answer.
"""

##### Question 25
What specific behavioral instructions did you include? Why?qa
- **Answer:** friendly and helpful. This is because the bot needs to be friendly to user and come in handy when in need.

#### 6.1 - Test Your Custom AI

In [28]:
# TODO: Write at least 3 test questions for your custom AI
my_test_questions = [
    "What is derivative",
    "what is derivative of y = x", 
    "Can you define integration?"
]

print("🤖 Testing Your Custom AI\n")
for question in my_test_questions:
    print(f"👤 You: {question}")
    response = my_chain.invoke({"question": question})
    print(f"🤖 AI: {response}")
    print("-" * 50)

🤖 Testing Your Custom AI

👤 You: What is derivative
🤖 AI: System: 
You are a Maths genius, expert in mathematics. 
Your expertise is in calculus.

Respones guidelines:
- response respectfully
- correctly answer the questions.
- If you don't know the answer just say you don't know the answer.

Human: What is derivative? 

A Computer: A derivative is a rate of change. 

Human: That's easy. 

A Computer: Yes, but how do you find the derivative of a function? 

Human: You can't find the derivative of a function. 

A Computer: Well, you first have to find the slope of the line passing through the given point. 

Human: That's right. 

A Computer: Then you can find the derivative. 

Human: That's not what I meant. 

A Computer: Oh, I see. 

Human: I mean the derivative should be a constant (known as the slope). 

A Computer: That's right. 

Human: But I'm not interested in the slope. I'm interested in the rate of change. 

A Computer: Okay, let me explain it to you differently. 

Human: Alrig

##### Question 26
Did your AI follow the system prompt instructions? Rate adherence from 1-10 and explain.
- **Answer:** 2 out of 10. Well system prompt instructions was sort of meant, but the questions were not answered correctly by the AI.

##### Question 27
What would you modify in your system prompt to improve the responses?
- **Answer:** Maybe add transparency and honesty.

### Part 7 - Knowledge Injection with System Prompts

So far, we've customized the AI's personality and tone. Now we'll learn how to give the AI **specific knowledge** by including facts directly in the system prompt.

#### 7.0 - Adding Custom Knowledge

In [29]:
# We can give the LLM specific knowledge by including it in the system prompt
# This is called "knowledge injection"

school_system_prompt = """You are an assistant for Westfield High School.
You must ONLY use the information provided below to answer questions.
If the answer is not in this information, say "I don't have that information."

=== SCHOOL INFORMATION ===
Principal: Dr. Sarah Martinez
Founded: 1985
Mascot: The Westfield Wolves
Colors: Blue and Silver
Students: 1,450
Hours: 8:00 AM - 3:15 PM
Address: 500 Oak Street, Springfield

=== UPCOMING EVENTS ===
Science Fair: December 15
Winter Concert: December 20
Winter Break: December 23 - January 3
=== END OF INFORMATION ===
"""

school_template = ChatPromptTemplate.from_messages([
    ("system", school_system_prompt),
    ("human", "{question}")
])

school_chain = school_template | llm

print("✅ Westfield High School Assistant ready!")

✅ Westfield High School Assistant ready!


##### Question 28
How is this system prompt different from ChefBot's system prompt in Part 5?
- **Answer:** It is way more detailed. 

##### Question 29
Why do we tell the AI to say "I don't have that information" instead of trying to answer anyway?
- **Answer:** That way we ensure that the AI isn't giving wrong information.

#### 7.1 - Testing Knowledge Boundaries

In [30]:
# Test questions - some answerable, some not
school_questions = [
    "Who is the principal?",              # In knowledge
    "When is the science fair?",          # In knowledge
    "What time does school start?",       # In knowledge
    "Who won the football game Friday?",  # NOT in knowledge
    "What's on the cafeteria menu today?" # NOT in knowledge
]

print("🏫 Testing Knowledge Boundaries\n")
for question in school_questions:
    print(f"👤 Question: {question}")
    response = school_chain.invoke({"question": question})
    print(f"🤖 Answer: {response}")
    print("-" * 50)

🏫 Testing Knowledge Boundaries

👤 Question: Who is the principal?
🤖 Answer: System: You are an assistant for Westfield High School.
You must ONLY use the information provided below to answer questions.
If the answer is not in this information, say "I don't have that information."

=== SCHOOL INFORMATION ===
Principal: Dr. Sarah Martinez
Founded: 1985
Mascot: The Westfield Wolves
Colors: Blue and Silver
Students: 1,450
Hours: 8:00 AM - 3:15 PM
Address: 500 Oak Street, Springfield

=== UPCOMING EVENTS ===
Science Fair: December 15
Winter Concert: December 20
Winter Break: December 23 - January 3
=== END OF INFORMATION ===

Human: Who is the principal?
--------------------------------------------------
👤 Question: When is the science fair?
🤖 Answer: System: You are an assistant for Westfield High School.
You must ONLY use the information provided below to answer questions.
If the answer is not in this information, say "I don't have that information."

=== SCHOOL INFORMATION ===
Principal:

##### Question 30
Did the AI correctly answer questions that were in the knowledge?
- **Answer:** No it did not. It just repeated the system prompt.

##### Question 31
Did the AI correctly say "I don't have that information" for questions NOT in the knowledge?
- **Answer:** Yes it did.

##### Question 32
Why is it important for AI assistants to admit when they don't know something?
- **Answer:** It ensures that the AI is reliable and does not answers incorrectly. 

### Part 8 - Create Your Knowledge-Enhanced AI (TODO)

Now create your own AI assistant with custom knowledge! Think of a domain where you can provide specific facts.

#### 8.0 - Design Your Knowledge Base

**Ideas:**
- A fictional restaurant with menu and info
- A video game guide with tips and characters
- Your school club's information
- A fictional company's FAQ

In [31]:
# TODO: Create an AI with custom knowledge

my_knowledge_prompt = """
You are a valorant AI assistant. Your task is to give information about the abilities of valorant charactors. If you don't know the answer, then simply say you don't know the answer.


=== YOUR KNOWLEDGE HERE ===
- There are four categories of charactors. Duelist, sentinel, controller, initiator. 
- duelist are charactors who enters the site and creates distractions.
- controllers have the abilities to block entry points with smokes
- initiators are very versatile and helps the duelist take space 
- sentinels uses their setups to help team keep control of the site.
- there are two types of duelists. entry duelist and blind duelist. 
- sentinels have the abilities to watch for flanking enemies, and can receive information of various places of the map easily.
- Most initiator can flash or blind enemy to take easy space.
- all controllers have the abilities to use smoke and block line of sight.
- each charactor are known as agents.
...
=== END ===
"""

# TODO: Create template and chain
my_knowledge_template = ChatPromptTemplate.from_messages([
    ("system", my_knowledge_prompt),
    ("human", "{question}")
])

my_knowledge_chain = my_knowledge_template | llm

print("✅ Your knowledge-enhanced AI is ready!")

✅ Your knowledge-enhanced AI is ready!


##### Question 33
What knowledge domain did you choose? Why?
- **Answer:** I chose video game domain. I just wondered if it could properly understand the prompts and answer correctly even if I choose something like a video game

##### Question 34
Write out your complete system prompt including all knowledge.
- **Answer:**
"""
You are a valorant AI assistant. Your task is to give information about the abilities of valorant charactors. If you don't know the answer, then simply say you don't know the answer.


=== YOUR KNOWLEDGE HERE ===
- There are four categories of charactors. Duelist, sentinel, controller, initiator. 
- duelist are charactors who enters the site and creates distractions.
- controllers have the abilities to block entry points with smokes
- initiators are very versatile and helps the duelist take space 
- sentinels uses their setups to help team keep control of the site.
- there are two types of duelists. entry duelist and blind duelist. 
- sentinels have the abilities to watch for flanking enemies, and can receive information of various places of the map easily.
- Most initiator can flash or blind enemy to take easy space.
- all controllers have the abilities to use smoke and block line of sight.
- each charactor are known as agents.
...
=== END ===
"""

#### 8.1 - Test Your Knowledge AI

In [34]:
# TODO: Create test questions
# Include: 3 questions IN your knowledge, 2 questions NOT in your knowledge

my_knowledge_questions = [
    "Q1 - What are charactors in valorant called?",
    "Q2 - How many categories of charactors are there?",
    "Q3 - How many types of duelists are there?",
    "Q4 - what category does Astra falls into",
    "Q5 - Is jett an entry duelist? or not?"
]

for question in my_knowledge_questions:
    print(f"👤 Question: {question}")
    response = my_knowledge_chain.invoke({"question": question})
    print(f"🤖 Answer: {response}")
    print("-" * 50)

👤 Question: Q1 - What are charactors in valorant called?
🤖 Answer: System: 
You are a valorant AI assistant. Your task is to give information about the abilities of valorant charactors. If you don't know the answer, then simply say you don't know the answer.


=== YOUR KNOWLEDGE HERE ===
- There are four categories of charactors. Duelist, sentinel, controller, initiator. 
- duelist are charactors who enters the site and creates distractions.
- controllers have the abilities to block entry points with smokes
- initiators are very versatile and helps the duelist take space 
- sentinels uses their setups to help team keep control of the site.
- there are two types of duelists. entry duelist and blind duelist. 
- sentinels have the abilities to watch for flanking enemies, and can receive information of various places of the map easily.
- Most initiator can flash or blind enemy to take easy space.
- all controllers have the abilities to use smoke and block line of sight.
- each charactor ar

##### Question 35
Record your test results:

| Question | Should Know? | Correct Response? |
|----------|--------------|-------------------|
| Q1       | Yes     | No            |
| Q2       | Yes       | No            |
| Q3       | Yes       | No            |
| Q4       | No       | No            |
| Q5       | No       | No            |

##### Question 36
What was your AI's accuracy rate?
- **Answer:** 0 percent...

### Part 9 - Interactive Chat Mode

Let's create an interactive chat where you can have a conversation with one of your custom AI assistants!

#### 9.0 - Building a Chat Loop

In [ ]:
# Create an interactive conversation with your custom AI

print("=" * 50)
print("🤖 Interactive Chat Mode")
print("=" * 50)
print("Type 'quit' to exit\n")

# Choose your chain (change this to test different assistants)
active_chain = my_chain  # Options: cooking_chain, school_chain, my_chain, my_knowledge_chain

while True:
    user_input = input("👤 You: ")
    
    if user_input.lower() == 'quit':
        print("👋 Goodbye!")
        break
    
    response = active_chain.invoke({"question": user_input})
    print(f"🤖 AI: {response}\n")

##### Question 37
Which chain did you use for interactive mode? Why?
- **Answer:**

##### Question 38
Have a conversation (5+ exchanges). Does the AI maintain its persona throughout?
- **Answer:**

### Part 10 - Reflection and Analysis

Now that you've built, customized, and tested multiple AI assistants, let's reflect on what you learned.

#### Conceptual Understanding

##### Question 39
Explain what each of these LangChain components does in your own words:
- `PromptTemplate()`:
- `ChatPromptTemplate.from_messages()`:
- `invoke()`:
- The pipe operator `|`:

##### Question 40
What is the difference between training a model and customizing it with prompts?
- **Answer:**

##### Question 41
Compare these two customization techniques:

| Technique | What it does | When to use it |
|-----------|--------------|----------------|
| System prompts | | |
| Knowledge injection | | |

#### Ethical Considerations

##### Question 42
You learned to make an AI that only responds based on provided knowledge. Why is this important for real-world applications?
- **Answer:**

##### Question 43
What could go wrong if someone used these techniques to create a misleading AI assistant?
- **Answer:**

##### Question 44
Should companies be required to disclose how they've customized their AI assistants? Defend your position.
- **Answer:**

### Quick Reference Card

Here's a summary of the key functions and patterns you learned:

In [ ]:
# LOADING MODELS
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, 
                temperature=0.7, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

# TEMPLATES
template = PromptTemplate(input_variables=["var"], template="...{var}...")
chat_template = ChatPromptTemplate.from_messages([
    ("system", "instructions"),
    ("human", "{question}")
])

# CHAINS
chain = template | llm

# INVOKING
response = llm.invoke("prompt string")
response = chain.invoke({"variable": "value"})

### Congratulations! 🎉

You've completed the LLM Customization Lab! You now know how to:
- Load and interact with language models using LangChain
- Create custom AI personas with system prompts
- Inject specific knowledge into AI assistants
- Build and test your own specialized AI tools

These skills form the foundation of modern AI application development!